In [12]:
# Import libraries
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import HTML, Markdown, display

# Set up secret credentials
from dotenv import load_dotenv 
load_dotenv()
import sys
import os

In [ ]:
# Error handling and retry logic not to worry about per-minute quota
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [8]:
# Set up your Google API key imported from .env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini client
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# Fetch the data for prompting
import requests
!wget -nv -O ecb.pdf https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/shared/pdf/ecb.ds250417~e613e58d41.es.pdf
document_file = client.files.upload(file='ecb.pdf')

2025-04-24 12:11:01 URL:https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/shared/pdf/ecb.ds250417~e613e58d41.es.pdf [140000] -> "ecb.pdf" [1]


PROMPTING 


1) *Zero-shot prompting* with parameters set for a slight creativity
2) *Few-shot prompting* with parameters set for a strict facticity
3) *Chain-of-thought prompting* with parameters set for the out-of-the-box ideas


In [13]:
# Zero-shot prompting
request = 'Tell me about the economic policy here.'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.2, top_p=0.95, top_k=30)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Here's a summary of the economic policy outlined in the document:

**Key Points:**

*   **Interest Rate Cut:** The European Central Bank (ECB) has decided to lower its three key interest rates by 25 basis points. This includes lowering the deposit facility rate to 2.25%, the main refinancing operations rate to 2.40%, and the marginal lending facility rate to 2.65%. The change is effective from April 23, 2025.

*   **Inflation Target:** The ECB's primary goal is to ensure that inflation stabilizes sustainably at its 2% medium-term target.

*   **Data-Dependent Approach:** Due to exceptional uncertainty, the ECB will take a data-dependent and meeting-by-meeting approach to determine the appropriate monetary policy stance. Decisions will be based on the inflation outlook, economic and financial data, underlying inflation dynamics, and the strength of monetary policy transmission.

*   **No Pre-Commitment:** The ECB is not pre-committing to a particular rate path.

*   **Asset Purchase Programs:** The Asset Purchase Programme (APP) and Pandemic Emergency Purchase Programme (PEPP) portfolios are declining as the Eurosystem no longer reinvests principal payments from maturing securities.

*   **Flexibility:** The Governing Council stands ready to adjust all its instruments to ensure inflation stabilizes at 2% and to preserve the smooth functioning of monetary policy transmission. The Transmission Protection Instrument is available to counter disorderly market dynamics.

**Economic Assessment:**

*   **Inflation:** The disinflation process is on track, with both headline and core inflation declining. Most measures suggest inflation will settle around the 2% target. Wage growth is moderating.
*   **Economic Growth:** The euro area economy has shown some resilience, but the outlook for growth has deteriorated due to rising trade tensions. Increased uncertainty is likely to reduce confidence and tighten financing conditions.
*   **Risks:** Downside risks to economic growth have increased due to trade tensions, geopolitical tensions, and potential financial market instability. Upside risks include increased defense and infrastructure spending.

**Other Considerations:**

*   **Fiscal and Structural Policies:** The document emphasizes the importance of fiscal and structural policies to make the euro area economy more productive, competitive, and resilient.
*   **Digital Euro:** The document mentions the importance of establishing a legislative framework for the potential introduction of a digital euro.

In essence, the ECB is taking a cautious approach, lowering interest rates to support the economy while remaining vigilant about inflation. They are emphasizing flexibility and data-driven decision-making in the face of global uncertainty.

In [14]:
# Few-shot prompting
request = '''1. What is the ECB's stance on inflation in the latest report?
            2. Describe the ECB’s recent actions related to monetary policy.
            3. Summarize the ECB's view on economic growth in the euro area.
            Tell me about the economic policy here.'''

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.1, top_p=0.9, top_k=20)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Here's a summary of the ECB's economic policy based on the provided document:

1.  **ECB's Stance on Inflation:**
    *   The ECB believes the disinflation process is "well on track."
    *   Both headline and core inflation declined in March.
    *   Most measures of underlying inflation suggest it will settle around the Governing Council's 2% medium-term target on a sustained basis.
    *   The ECB is determined to ensure inflation stabilizes sustainably at its 2% medium-term target.

2.  **ECB's Recent Actions Related to Monetary Policy:**
    *   The Governing Council decided to lower the three key ECB interest rates by 25 basis points.
    *   Specifically, the deposit facility rate was lowered, which is the rate through which the Governing Council steers the monetary policy stance.
    *   The interest rates on the deposit facility, the main refinancing operations, and the marginal lending facility will be decreased to 2.25%, 2.40%, and 2.65% respectively, with effect from 23 April 2025.
    *   The APP and PEPP portfolios are declining at a measured and predictable pace, as the Eurosystem no longer reinvests the principal payments from maturing securities.

3.  **ECB's View on Economic Growth in the Euro Area:**
    *   The euro area economy has been building up some resilience against global shocks.
    *   The outlook for growth has deteriorated owing to rising trade tensions.
    *   Increased uncertainty is likely to reduce confidence among households and firms.
    *   The adverse and volatile market response to the trade tensions is likely to have a tightening impact on financing conditions.
    *   These factors may further weigh on the economic outlook for the euro area.
    *   The economy is likely to have grown in the first quarter of the year, and manufacturing has shown signs of stabilisation.

**Economic Policy Summary:**

The ECB's economic policy is focused on achieving its 2% medium-term inflation target while supporting economic growth in the euro area. The current policy stance involves:

*   **Lowering Interest Rates:** To stimulate economic activity and support the disinflation process.
*   **Data-Dependent Approach:** The ECB will make decisions on a meeting-by-meeting basis, based on incoming economic and financial data, the dynamics of underlying inflation, and the strength of monetary policy transmission.
*   **Flexibility:** The ECB stands ready to adjust all of its instruments within its mandate to ensure that inflation stabilizes sustainably at its 2% target and to preserve the smooth functioning of monetary policy transmission.
*   **Addressing Trade Tensions:** The ECB acknowledges that rising trade tensions pose a risk to economic growth and is monitoring the situation closely.
*   **Fiscal and Structural Policies:** The ECB emphasizes the importance of fiscal and structural policies to make the euro area economy more productive, competitive, and resilient.


In [15]:
# Chain-of-thought prompting
request = "What is the ECB's forecast for economic growth? Break it down into steps and think through the logic behind the ECB's predictions"

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.9, top_p=0.99, top_k=40)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Based on the provided text from the ECB's statement (dated April 17, 2025), it doesn't contain a specific numerical forecast for economic growth, but it does provide insights into their assessment and the factors they are considering.

Here's a breakdown of how to interpret the ECB's outlook:

**1. Overall Assessment:**

*   **Growth Outlook Deteriorated:** The ECB clearly states the outlook for growth has deteriorated due to rising trade tensions. This implies they expect slower growth than previously anticipated.
*   **Resilience Against Shocks:**  The euro area economy has been building up some resilience against global shocks (as stated in the document).
*   **Likely Grew in Q1:** The document mentions that the economy is likely to have grown in the first quarter of the year.

**2. Factors Weighing on Growth:**

*   **Trade Tensions:**  Increased global trade tensions are a key concern, creating uncertainty and dampening exports.
*   **Uncertainty:** Increased uncertainty, likely due to trade tensions and geopolitical events, reduces confidence among households and firms, leading to less spending and investment.
*   **Tightening Financing Conditions:** Adverse reactions to the trade tensions are likely to tighten financing conditions, making it more expensive for businesses to borrow and invest.
*   **Disruptions to International Commerce:** Disruptions to international commerce is weighing on business investment.
*   **Geopolitical Uncertainty:** Geopolitical tensions, such as Russia's unjustified war against Ukraine and the tragic conflict in the Middle East, also remain a major source of uncertainty.

**3. Factors Supporting Growth:**

*   **Strong Labor Market:** A strong labor market with low unemployment is seen as a support for spending.
*   **Higher Real Incomes:**  Higher real incomes (likely due to wages rising faster than inflation) should support spending.
*   **Monetary Policy Impact:**  Previous (and potentially future) monetary policy easing (interest rate cuts) is expected to stimulate spending.
*   **Defense and Infrastructure Spending:** Increased defense and infrastructure investments at national and EU levels should bolster manufacturing.

**4. Risks to the Outlook:**

*   **Downside Risks Increased:** The ECB acknowledges that downside risks to economic growth have increased.
*   **Trade Tensions:** The escalation of trade tensions is seen as a significant downside risk, potentially dampening exports, investment, and consumption.

**5. Key Considerations and Actions by the ECB**

*   **Data Dependent:** Given the exceptional uncertainty, the ECB will take a "data-dependent and meeting-by-meeting approach" to monetary policy decisions.
*   **Inflation Outlook:** Interest rate decisions will be based on the assessment of the inflation outlook, underlying inflation dynamics, and the strength of monetary policy transmission.
*   **Not Pre-committing:** The ECB is not pre-committing to a particular interest rate path.
*   **Adjust All Instruments:** The Governing Council stands ready to adjust all of its instruments within its mandate to ensure that inflation stabilises sustainably at its 2% target over the medium term and to preserve the smooth functioning of monetary policy transmission.

**In summary, while the ECB doesn't give a precise GDP growth percentage, the document leans toward a cautious outlook. They signal slower growth than previously expected due to trade tensions and uncertainty.** They highlight some factors that could provide support, but emphasize the increased downside risks. The ECB's actions and statements indicate a readiness to act if the economic or inflation situation deteriorates.
